In [1]:
# # Cài đặt RAPIDS
# !conda install -c rapidsai -c nvidia -c conda-forge \
#     cudf=23.10 cuml=23.10 python=3.10 -y

# Install RAPIDS using pip
!pip install cudf-cu12 cuml-cu12 rmm-cu12 --extra-index-url=https://pypi.nvidia.com
!pip install numpy==1.23.5  # Downgrade NumPy to be compatible with Numba

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - nodefaults
 - defaults
 - pytorch
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - cudf=23.10
    - cuml=23.10
    - python=3.10


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aws-c-auth-0.7.4           |       hc8144f4_1          99 KB  conda-forge
    aws-c-cal-0.6.2            |       h09139f6_2          50 KB  conda-forge
    aws-c-common-0.9.3         |       hd590300_0         215 KB  conda-forge
    aws-c-compression-0.2.17   |       h184a658_3          19 KB  conda-forge
    aws-c-event-stream-0.3.2   |       hd6ebb48_1          52 KB  conda-forge
    aws-c-http-0.7.13          |       hc690213_1         189 KB  conda-forge
    aws-c-io-0.13.32           |       h161b759_6         150 KB  conda-forge
    aw

In [3]:
!pip install numpy==1.24

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 80.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.10.2 requires cubinlinker, which is not installed.
cudf 23.10.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.10.2 requires ptxcompiler, which is not installed.
cuml 23.10.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.10.2 requires cupy-cuda11x>=12.0.0, which is not installed.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.24.0 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you ha

In [5]:
!nvidia-smi

Sun Nov 24 13:00:30 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P0             33W /  250W |     257MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

 Cài đặt RAPIDS cho CUDA 12.6
Sử dụng lệnh sau để cài đặt RAPIDS với CUDA 12.6:

```bash
conda install -n rapids_env -c rapidsai -c nvidia -c conda-forge \
    cudf=23.10 cuml=23.10 python=3.10 cupy-cuda12x -y
```

In [ ]:
# # Cài đặt RAPIDS
# !conda install -c rapidsai -c nvidia -c conda-forge \
#     cudf=23.10 cuml=23.10 python=3.10 -y

!conda install -n rapids_env -c rapidsai -c nvidia -c conda-forge \
    cudf=23.10 cuml=23.10 python=3.10 cupy-cuda12x -y


In [ ]:
!conda install numpy=1.24.3 -y

Channels:
 - rapidsai
 - nvidia
 - nodefaults
 - conda-forge
 - defaults
 - pytorch
Platform: linux-64
Solving environment: - 

In [4]:
import json
import cudf
import cupy as cp
from cuml.decomposition import TruncatedSVD

# Đường dẫn đến file dữ liệu
train_file = '/kaggle/input/otto-recommender-system/train.jsonl'
test_file = '/kaggle/input/otto-recommender-system/test.jsonl'

# Bước 1: Đọc và xử lý dữ liệu train
print("Đang đọc và xử lý dữ liệu train...")
sessions = []
products = []
interactions = []

# Đọc dữ liệu và chỉ sử dụng số lượng sessions giới hạn (ví dụ: 1000)
max_sessions = 1000  # Tăng giảm tùy thuộc vào yêu cầu và tài nguyên
session_count = 0

with open(train_file, 'r') as file:
    for line in file:
        if session_count >= max_sessions:
            break
        data = json.loads(line)
        session_id = data['session']
        session_count += 1
        for event in data['events']:
            aid = event['aid']
            event_type = event['type']
            # Gán trọng số dựa trên loại hành động
            weight = 1 if event_type == 'clicks' else (3 if event_type == 'carts' else 5)
            sessions.append(session_id)
            products.append(aid)
            interactions.append(weight)

# Tạo DataFrame từ dữ liệu thu thập
interaction_df = cudf.DataFrame({'session': sessions, 'product': products, 'interaction': interactions})
print(f"Đã xử lý {session_count} sessions.")

# Lọc chỉ giữ lại top 2000 sản phẩm phổ biến nhất
top_products = interaction_df['product'].value_counts().iloc[:200].index
interaction_df_filtered = interaction_df[interaction_df['product'].isin(top_products)]

# Tạo ma trận sparse từ dữ liệu đã giảm kích thước
interaction_matrix = interaction_df_filtered.pivot_table(index='session', columns='product', values='interaction', fill_value=0)

# Chuyển ma trận sang dạng sparse trên GPU
interaction_sparse = cp.sparse.csr_matrix(interaction_matrix.values)
print(f"Kích thước ma trận sau lọc: {interaction_sparse.shape}")

# Bước 2: Áp dụng Truncated SVD
print("Đang áp dụng SVD...")
svd = TruncatedSVD(n_components=2, random_state=42)  # Số yếu tố ẩn
latent_matrix = svd.fit_transform(interaction_sparse)

# Tái tạo ma trận dự đoán
reconstructed_matrix = cp.dot(latent_matrix, svd.components_)
print("Ma trận dự đoán đã được tạo.")

# Bước 3: Đọc và xử lý dữ liệu test
print("Đang đọc và xử lý dữ liệu test...")
predictions = []

with open(test_file, 'r') as file:
    for line in file:
        data = json.loads(line)
        session_id = data['session']
        # Kiểm tra nếu session tồn tại trong tập train
        if session_id in interaction_matrix.index:
            # Lấy top 20 sản phẩm dự đoán
            predicted_products = reconstructed_matrix[interaction_matrix.index.get_loc(session_id), :].argsort()[::-1][:20]
        else:
            predicted_products = top_products[:20].to_array()
        
        # Tạo dự đoán cho các loại hành động
        predictions.append(f"{session_id}_clicks," + " ".join(map(str, predicted_products)))
        predictions.append(f"{session_id}_carts," + " ".join(map(str, predicted_products)))
        predictions.append(f"{session_id}_orders," + " ".join(map(str, predicted_products)))

# Bước 4: Ghi kết quả ra file submission.csv
output_file = '/kaggle/working/submission.csv'
print(f"Đang ghi kết quả ra file {output_file}...")

with open(output_file, 'w') as f:
    f.write("session_type,labels\n")
    for line in predictions:
        f.write(line + "\n")

print("Dự đoán hoàn tất! File submission.csv đã được tạo.")


ImportError: Numba needs NumPy 1.24 or less